In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import matthews_corrcoef
import numpy as np

# 1. One hot Encoding
This section defines the helper functions required to transform raw protein sequences into numerical matrices suitable for the Neural Network.

The process involves two main steps:
1.  **Sequence Standardization:** All sequences are fixed to a length of **90 residues**. Sequences longer than 90 are truncated, while shorter ones are padded using the character 'X'.
2.  **Encoding with Explicit Padding:** We use a One-Hot Encoding scheme with **21 dimensions** (20 standard amino acids + 1 for the padding token 'X'). This allows the model to explicitly distinguish padding from actual biological data.

In [1]:
import pandas as pd
import numpy as np

def create_one_hot_sets(dataset):
    """
    Processes the dataset by splitting it into predefined sets, 
    standardizing sequence lengths, and applying One-Hot Encoding. 
    """
    # Define the specific dataset partitions (folds) to process
    sets = ["1","2","3","4","5"]
    set_results=[]
    for i in sets:
        # Filter the dataframe to retrieve only rows belonging to the current set ID
        st=dataset.query(f"Set == '{i}'")
        tmp_x=[]
        tmp_y=[]
        # Iterate through each biological sequence in the current set
        for _ , row in st.iterrows():
            
            # --- Sequence Length Standardization (Fixed Length: 90) ---
            # If the sequence is shorter than 90, add padding (using 'X' which becomes zeros)
            seq=row["Sequence"]
            if len(seq) < 90:
                seq=_increase_lenseq(seq)
            # If the sequence is longer than 90, truncate it (keep N-terminus info)
            else:
                seq=seq[:90]

            # --- Feature Extraction ---
            # Convert the standardized string sequence into a One-Hot matrix (90x20)
            encoded_seq = one_hot_encoding(seq)
            tmp_x.append(encoded_seq)

            # --- Label Encoding ---
            # Convert string labels into binary format (1 for Signal Peptide, 0 for others)
            if row["Class"] == "Positive":
                y=1
            else:
                y=0
            tmp_y.append(y)
        tmp_x=np.array(tmp_x, dtype=np.float32)
        tmp_y=np.array(tmp_y, dtype=np.float32)
        set_results.append((tmp_x , tmp_y))
    return set_results
            
def _increase_lenseq(seq):
    """
    Adds padding to the sequence to reach the target length of 90.
    """
    x=len(seq)
    num_of_X= 90-x
    # Append 'X' characters which will be encoded as zero-vectors later
    seq=seq+("X"*num_of_X)
    return seq
            
def one_hot_encoding(sequence):
    """
    Transforms an amino acid sequence into a binary matrix.
    Output shape: (Sequence_Length, 21)
    """
    M = []
    # Standard 20 amino acids alphabet + 1 the X unknown amino acid
    aa_alph = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y','X']
    for aa in sequence:
        one_hot = np.zeros(21)
        try:
            # Find index of the AA (including 'X') and put 1 for that position
            index = aa_alph.index(aa)
            one_hot[index] = 1
        except:
            pass
        M.append(one_hot)
    M = np.array(M)
    return M

 # 2. Network definition and infrastructure preparation

 ## 2.1. Neural Network Architecture: Hybrid CNN-LSTM

We implement a hybrid Deep Learning architecture designed to capture both local structural motifs and global sequential patterns in protein sequences.

The architecture consists of three main blocks:
2.1.1.  **Convolutional Block (CNN):** Uses a 1D convolution with a kernel size of 17 (approx. length of a Signal Peptide hydrophobic region) to extract local biochemical features (motifs) from the One-Hot encoded sequence.
2.1.2.  **Recurrent Block (LSTM):** Processes the feature map generated by the CNN to model sequential dependencies and long-range interactions along the amino acid chain.
2.1.3.  **Classification Head (MLP):** A dynamic Multi-Layer Perceptron that takes the final hidden state of the LSTM and outputs the probability of the sequence being a Signal Peptide.

## 2.2. Infrastructure preparation for further utility
This section implements also the essential infrastructure to train, validate, and test the model efficiently, addressing memory constraints and training stability.

### 2.2.1. Custom Dataset: `SignalDataset`
To handle the large memory footprint of One-Hot encoded sequences, we implement a **Lazy Loading** strategy:
* **Memory Efficiency:** The class stores only references to the raw NumPy arrays rather than creating tensor copies immediately.
* **On-the-fly Conversion:** Data is converted into PyTorch Tensors only within the `__getitem__` method, exactly when the DataLoader requests a specific batch.
* **Label Shaping:** It ensures target labels are reshaped to `[1]` to ensure compatibility with the `BCELoss` function.

### 2.2.2. Training Engine: `train_val`
The main loop that orchestrates the learning process. It includes several stability mechanisms:
* **Gradient Clipping:** We apply `torch.nn.utils.clip_grad_norm_(max_norm=1.0)` to prevent the **Exploding Gradient** problem (common in LSTMs), ensuring stable weight updates.
* **Early Stopping:** The training monitors the Validation MCC. If performance stops improving for a defined number of epochs (`patience`), training terminates early to prevent **Overfitting**.
* **Best Model Tracking:** The function automatically saves and returns the weights of the *best* performing epoch, ensuring we don't return an overfitted model from the final epoch.

### 2.2.3. Inference: `test`
A lightweight function for evaluating the model on the Test or Benchmark sets:
* **Evaluation Mode:** Sets `model.eval()` to disable stochastic layers like Dropout and freeze Batch Normalization stats.
* **Resource Optimization:** Uses `torch.no_grad()` to skip gradient computation, reducing memory usage and speeding up inference.

### 2.2.4. Hardware Acceleration
The code automatically detects available hardware (`torch.cuda.is_available()`) to offload computations to the **GPU** for significantly faster training times.

In [3]:

class SP_NN(nn.Module):
    def __init__(self, input_size, hidden_sizes,lstm_hidden_size, num_lstm_layers, output_size, dropout_p=0.5):
        """
        Hybrid CNN-LSTM model for Signal Peptide prediction.
        
        Args:
            input_size (int): Dimension of the input vector (21 for Explicit Padding One-Hot).
            hidden_sizes (list): List of integers defining the architecture of the MLP head.
            lstm_hidden_size (int): Number of features in the LSTM hidden state.
            num_lstm_layers (int): Number of stacked LSTM layers.
            output_size (int): Size of the output (1 for binary classification).
            dropout_p (float): Dropout probability for regularization.
        """
        super(SP_NN, self).__init__()

    
        # --- 1. Convolutional Block (Local Feature Extraction) ---
        # We map the 21 input channels to 64 feature maps.
        # Kernel size 17 is chosen to cover typical secondary structure elements.
        self.cnn_out_channels = 64
        self.conv1 = nn.Conv1d(input_size, self.cnn_out_channels, kernel_size=17, padding='same')

        # --- 2. Recurrent Block (Sequential Processing) ---
        # The LSTM takes the 64 features from the CNN as input.
        self.lstm = nn.LSTM(self.cnn_out_channels, lstm_hidden_size, num_lstm_layers, 
                            batch_first=True, dropout=dropout_p if num_lstm_layers > 1 else 0)
        
        # Batch Normalization to stabilize the LSTM output before the classifier
        self.bn = nn.BatchNorm1d(lstm_hidden_size)
        
        # --- 3. Dynamic MLP Head (Classification) ---
        mlp_layers = []
        current_input_size = lstm_hidden_size 
        
        # Dynamically build hidden layers based on the configuration chosen
        for hidden_size in hidden_sizes:
            # Linear Layer
            mlp_layers.append(nn.Linear(current_input_size, hidden_size))
            # Activation Function (ReLU introduces non-linearity)
            mlp_layers.append(nn.ReLU())
            # Regularization (Dropout prevents overfitting)
            mlp_layers.append(nn.Dropout(p=dropout_p))
            # Update input dimension for the next layer
            current_input_size = hidden_size
            
        # Final Output Layer
        mlp_layers.append(nn.Linear(current_input_size, output_size))
        
        # Sigmoid activation for binary output (0-1)
        mlp_layers.append(nn.Sigmoid()) 
        
        # Assemble all the individual layers into a single functional block.
        # This connects them in order, so data flows through them automatically.
        # (Without this, you would have to call each layer manually in the forward pass, 
        # which is impossible when the number of layers is dynamic).
        self.mlp = nn.Sequential(*mlp_layers)

    def forward(self, x):
        # Input x shape: [Batch, Length(90), Channels(21)]
        
        # --- 1. Convolutional Block ---
        # Conv1d needs input as [Batch, Features, Length]
        x = x.permute(0, 2, 1) 
        x = self.conv1(x)     
        # Output shape: [Batch, 64, 90]
        
        # --- 2. Recurrent Block (LSTM) ---
        # LSTM needs input as [Batch, Length, Features] -> Permute back
        x = x.permute(0, 2, 1)
        out, _ = self.lstm(x)

        # LSTM Output shape: [Batch, 90, Hidden_Size]
        
        # --- 3. Sequence Summarization ---
        # Select only the output of the last time step.
        # This summarizes the entire 90-AA sequence into a single vector.
        # Shape transformation: [Batch, 90, Hidden] -> [Batch, Hidden]
        out = out[:, -1, :] 
        out = self.bn(out)
        
        # --- 4. Classification Head ---
        # Normalize and pass through the dynamic MLP
        out = self.mlp(out)
        return out


class SignalDataset(Dataset):
    """
    Custom Dataset class designed for memory efficiency ('Lazy Loading').
    It converts data to Tensors only when needed, preventing RAM saturation.
    """
    def __init__(self, X, y):
        # Store references to the raw data (NumPy arrays).
        # CRITICAL: We do NOT convert to Tensors here to avoid duplicating memory usage
        # immediately. The data remains in its original, lighter format until requested.
        self.X = X 
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Convert only the requested sample (at index 'idx') to a PyTorch Tensor.
        # This keeps the memory footprint low during training.
        x_out = torch.from_numpy(self.X[idx]).float() 
        
        # Transform the scalar label (e.g., 0) into a vector (e.g., [0]).
        # Using .view(1) is necessary to match the model's output shape [Batch_Size, 1]
        # and avoid dimension mismatch errors with the Loss Function (BCELoss)
        y_out = torch.tensor(self.y[idx], dtype=torch.float32).view(1)
        
        return x_out, y_out


def train_val(model, 
              train_loader, 
              val_loader,
              optimizer, 
              criterion, 
              epochs, 
              patience,
              scorer = matthews_corrcoef,
              init_best_score = -1,
              output_transform = lambda x: (x > 0.5).float()):
"""
    Executes the training and validation loop with Early Stopping and Gradient Clipping.

    Args:
        model (nn.Module): The PyTorch neural network to train (e.g., SignalNet).
        train_loader (DataLoader): DataLoader containing the training batches.
        val_loader (DataLoader): DataLoader containing the validation batches.
        optimizer (torch.optim.Optimizer): Algorithm to update model weights (e.g., Adam).
        criterion (nn.Module): Loss function to measure error (e.g., BCELoss).
        epochs (int): Maximum number of training epochs.
        patience (int): Number of epochs to wait for improvement before stopping early.
        scorer (callable, optional): Metric function to evaluate model performance (default: MCC).
        init_best_score (float, optional): Initial baseline score for model tracking.
        output_transform (callable, optional): Function to convert raw outputs to class labels.
    Returns:
        dict: The state_dict (weights) of the best performing model found during training.
    """
  # Initialize tracking variables for Early Stopping
  best_val_score = init_best_score 
  epochs_without_improvement = 0
  best_model_state_dict = None 

  for epoch in range(epochs): #ripeti il processo per epoche volte. 
    # ===========================
    #      TRAINING PHASE
    # ===========================
    # Set model to training mode (enables Dropout and BatchNorm updates)
      model.train()
      # Initialize accumulator for the epoch loss
      loss = 0
      for batch_X, batch_y in train_loader: 
         # Move data to GPU (if available), useful to speed up performances
          batch_X, batch_y = batch_X.to(device), batch_y.to(device)      
          # 1. Reset Gradients (Clear previous step's gradients)
          optimizer.zero_grad()    
          # 2. Forward Pass (Compute predictions)    
          outputs = model(batch_X)      
          # 3. Compute Loss (Compare predictions vs targets)
          loss = criterion(outputs, batch_y)        
          # 4. Backward Pass (Compute gradients) 
          loss.backward()         
          # 5. Gradient Clipping (CRITICAL for LSTMs)
          # Prevents "exploding gradients" by capping the norm to 1.0
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
          # 6. Optimizer Step (Update weights)
          optimizer.step() #prende i calcoli della backward e aggiorna fisicamente i pesi del cervello per ridurre l'errore

    # ===========================
    #     VALIDATION PHASE
    # ===========================
    # Set model to evaluation mode (disables Dropout)
      model.eval() 
      val_preds = []
      val_labels = []
      with torch.no_grad(): # Disable gradient calculation for validation (saves memory and computation)
          for batch_X, batch_y in val_loader: 
              batch_X, batch_y = batch_X.to(device), batch_y.to(device)
              outputs = model(batch_X)
              # Transform output probabilities to class labels (0 or 1)
              preds = output_transform(outputs) #utilizza il metodo di traformazione conenuto in output _transform permettendolo di variare a piacimento
              # Store predictions and labels for metric calculation
              val_preds.extend(preds.cpu().numpy().flatten()) #aggiunge le risposte alle liste
              val_labels.extend(batch_y.cpu().numpy().flatten())
      
      # Calculate validation metric (MCC)
      val_score = scorer(val_labels, val_preds) 
        # ===========================
        #      EARLY STOPPING
        # ===========================
        # Check if current model is better than the best found so far
      if val_score > best_val_score:
          best_val_score = val_score
          epochs_without_improvement = 0
          # Save the best model weights in memory
          best_model_state_dict = model.state_dict()
          print('Validation score improved to {:.4f}'.format(best_val_score))
      else:
          # No improvement: increase patience counter
          epochs_without_improvement += 1
          if epochs_without_improvement >= patience:
              print('Early stopping at epoch {}'.format(epoch+1))
              break # Stop training

      print('Epoch [{}/{}], Loss: {:.4f}, Val score: {:.4f}'.format(epoch+1, epochs, loss.item(), val_score))
  return best_model_state_dict # Return the best weights found during training

def test(model, test_loader, scorer = matthews_corrcoef, output_transform = lambda x: (x > 0.5).float()):
    """
    Evaluates the trained model on a test dataset without updating weights.

    Args:
        model (nn.Module): The trained PyTorch model to evaluate.
        test_loader (DataLoader): DataLoader containing the test dataset batches.
        scorer (callable, optional): Metric function to calculate the final score (default: matthews_corrcoef).
        output_transform (callable, optional): Function to convert raw model outputs (probabilities) 
            into binary class labels (default: threshold > 0.5).

    Returns:
        float: The calculated performance score (e.g., MCC) on the entire test set.
    """
  model.eval() # Set model to evaluation mode (disables Dropout and BatchNorm statistics updates)
  all_preds = []
  all_labels = []
  with torch.no_grad():
      for batch_X, batch_y in test_loader:
          batch_X, batch_y = batch_X.to(device), batch_y.to(device) # Move data to the configured device (GPU or CPU)
          outputs = model(batch_X) # Forward pass: compute predictions
          preds = output_transform(outputs) # Apply threshold to get binary class labels (e.g., > 0.5 -> 1)

          # Move tensors to CPU, convert to NumPy arrays, and flatten them
          # (Required for Scikit-learn metrics like MCC which work on CPU arrays)
          all_preds.extend(preds.cpu().numpy().flatten()) 
          all_labels.extend(batch_y.cpu().numpy().flatten())
 # Calculate the final performance score based on all predictions
  score = scorer(all_labels, all_preds) #MCC
  return score

# --- Device Configuration ---
# Check if a NVIDIA GPU with CUDA support is available on the system.
if torch.cuda.is_available():
    # Set the computation device to GPU.
    # All models and tensors moved to this device will use VRAM and hardware acceleration.
    device = torch.device("cuda")
    print("GPU is available")
else:
    # Fallback to CPU if no compatible GPU is found.
    # Training will work but will be significantly slower.
    device = torch.device("cpu")
    print("GPU is not available")

GPU is available


# 3. Hyperparameter Tuning & Model Optimization

In this section, we execute an automated **Hyperparameter Optimization (HPO)** pipeline to determine the optimal configuration for our hybrid **CNN-LSTM** architecture (`SignalPeptideLSTM`).

Given the complexity of the model and the biological variability of the signal peptides, manually selecting parameters would be inefficient. We leverage **Ray Tune** to explore the search space systematically.

### Methodology
To ensure robust model selection and computational efficiency, the pipeline incorporates:
1.  **5-Fold Cross-Validation:** Each configuration is trained and validated on 5 different splits of the data to avoid overfitting to a specific subset. The selection metric is the **Average MCC** across folds.
2.  **Memory Optimization:** To prevent *Out-Of-Memory (OOM)* errors with the One-Hot encoded dataset, we utilize **Ray's Shared Object Store**. Data is loaded once into shared memory and accessed by reference, minimizing RAM usage.
3.  **Resource Management:** We enforce sequential execution (1 trial at a time) to fully utilize the GPU without saturation.

### Hyperparameter Search Space
We tune a combination of architectural and training parameters:
* **Architecture (LSTM):** Number of stacked layers (`num_lstm_layers`) and hidden state dimension (`lstm_hidden_size`).
* **Architecture (MLP Head):** Dynamic depth (`num_layers`) and width (`hidden_sizes`) of the classification head, testing both "Funnel" and "Flexible" topologies.
* **Optimization:** Learning Rate (`lr`) and Batch Size (`batch_size`).
* **Regularization:** Dropout probability (`dropout`).

### Outcome
The goal of this section is to identify the **Best Trial Configuration**. These optimal hyperparameters will be retrieved and used to instantiate the final model for the definitive performance evaluation on the independent benchmark dataset (see `benchmark_test.ipynb`).

In [4]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import random
import ray
import gc

# --- Ray Initialization & Resource Management ---

# 1. Safety Shutdown
# Check if a Ray instance is already running (e.g., from a previous cell).
# If so, shut it down to ensure a clean slate and release locked resources
if ray.is_initialized():
    ray.shutdown()

# 2. Constrained Initialization
# Initialize Ray with a strict memory limit on the Object Store.
# We limit it to 2GB (2 * 1024^3 bytes) instead of the default (which can be ~30% of RAM).
# This prevents Ray from reserving too much RAM upfront, leaving space for 
# the dataset loading and the operating system, avoiding "OutOfMemory" crashes.
ray.init(object_store_memory=2*1024*1024*1024) 


2025-11-19 13:37:38,612	INFO worker.py:2012 -- Started a local Ray instance.
C:\Users\chari\.conda\envs\lab_final\lib\site-packages\ray\_private\worker.py:2051: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(


Python version:,3.10.19
Ray version:,2.51.1


(pid=gcs_server) [2025-11-19 13:38:05,940 E 33568 29984] (gcs_server.exe) gcs_server.cc:302: Failed to establish connection to the event+metrics exporter agent. Events and metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(test_config pid=27040) C:\Users\chari\AppData\Local\Temp\ipykernel_16604\1353098902.py:70: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_numpy.cpp:212.)


(test_config pid=27040) Validation score improved to 0.0000
(test_config pid=27040) Epoch [1/100], Loss: 0.6221, Val score: 0.0000
(test_config pid=27040) Epoch [2/100], Loss: 0.1730, Val score: 0.0000


(raylet) [2025-11-19 13:38:12,168 E 32732 31588] (raylet.exe) main.cc:975: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14


...

(test_config pid=26288) Epoch [8/100], Loss: 0.5619, Val score: 0.0000
(test_config pid=26288) Epoch [9/100], Loss: 0.0466, Val score: 0.0000
(test_config pid=26288) Epoch [10/100], Loss: 0.0973, Val score: 0.0000
(test_config pid=26288) Validation score improved to 0.2733
(test_config pid=26288) Epoch [11/100], Loss: 0.8946, Val score: 0.2733
(test_config pid=26288) Validation score improved to 0.3888
(test_config pid=26288) Epoch [12/100], Loss: 0.8062, Val score: 0.3888
(test_config pid=26288) Validation score improved to 0.5623
(test_config pid=26288) Epoch [13/100], Loss: 0.0168, Val score: 0.5623
(test_config pid=26288) Validation score improved to 0.6300
(test_config pid=26288) Epoch [14/100], Loss: 0.0543, Val score: 0.6300
(test_config pid=26288) Validation score improved to 0.6354
(test_config pid=26288) Epoch [15/100], Loss: 0.2033, Val score: 0.6354
(test_config pid=26288) Validation score improved to 0.7051
(test_config pid=26288) Epoch [16/100], Loss: 0.2443, Val score: 0

(test_config pid=14744) C:\Users\chari\AppData\Local\Temp\ipykernel_16604\1353098902.py:70: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_numpy.cpp:212.)


(test_config pid=14744) Validation score improved to 0.0000
(test_config pid=14744) Epoch [1/100], Loss: 0.1185, Val score: 0.0000
(test_config pid=14744) Epoch [2/100], Loss: 0.0891, Val score: 0.0000
(test_config pid=14744) Epoch [3/100], Loss: 0.1459, Val score: 0.0000
(test_config pid=14744) Epoch [4/100], Loss: 0.0884, Val score: 0.0000
(test_config pid=14744) Epoch [5/100], Loss: 0.5659, Val score: 0.0000
(test_config pid=14744) Epoch [6/100], Loss: 0.6539, Val score: 0.0000
(test_config pid=14744) Epoch [7/100], Loss: 0.9157, Val score: 0.0000
(test_config pid=14744) Epoch [8/100], Loss: 0.1369, Val score: 0.0000


(test_config pid=14744) [2025-11-19 16:26:17,991 E 14744 34900] core_worker_process.cc:825: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14


(test_config pid=14744) Validation score improved to 0.2644
(test_config pid=14744) Epoch [9/100], Loss: 0.1521, Val score: 0.2644
(test_config pid=14744) Epoch [10/100], Loss: 0.1440, Val score: -0.0088
(test_config pid=14744) Epoch [11/100], Loss: 0.2042, Val score: 0.2557
(test_config pid=14744) Validation score improved to 0.3790
(test_config pid=14744) Epoch [12/100], Loss: 0.1124, Val score: 0.3790
(test_config pid=14744) Validation score improved to 0.5079
(test_config pid=14744) Epoch [13/100], Loss: 0.1598, Val score: 0.5079
(test_config pid=14744) Validation score improved to 0.7022
(test_config pid=14744) Epoch [14/100], Loss: 1.0094, Val score: 0.7022
(test_config pid=14744) Validation score improved to 0.8601
(test_config pid=14744) Epoch [15/100], Loss: 0.0053, Val score: 0.8601
(test_config pid=14744) Epoch [16/100], Loss: 0.3109, Val score: 0.8516
(test_config pid=14744) Validation score improved to 0.9164
(test_config pid=14744) Epoch [17/100], Loss: 0.0018, Val score:

In [5]:
# --- Data Loading & Preprocessing ---
# Load the raw dataset from disk
dataset = pd.read_csv("../Data_Preparation/train_bench.tsv", sep="\t")

# Process the dataset: Standardization (length 90), One-Hot Encoding (21 channels),
# and splitting into the 6 predefined sets.
# Returns a list of tuples: [(X1, y1), (X2, y2), ... (X_bench, y_bench)]
all_data = create_one_hot_sets(dataset)
print("Spostamento dati nel Ray Object Store...")

# --- Memory Optimization: Shared Object Store ---
# Instead of passing the heavy 'all_data' variable to every worker (which causes memory 
# explosion and serialization errors), we store it ONCE in Ray's shared memory.
data_ref = ray.put(all_data)# 'ray.put' returns a lightweight reference ID ('data_ref') that acts as a retrieval ticket.

# --- Critical Cleanup ---
# We MUST delete the local variable 'all_data' from the global scope.
# Reason: This prevents Ray from accidentally "capturing" the huge variable 
# inside the training function definition (fixing the "ImplicitFunc is too large" error).
del all_data
gc.collect() # Force Python's Garbage Collector to release the RAM immediately
print("Dati archiviati. Riferimento creato.")

Spostamento dati nel Ray Object Store...
Dati archiviati. Riferimento creato.


In [6]:
def generate_hidden_layer_size(config):
    """
    Helper function for Ray Tune to generate dynamic MLP architectures.
    It samples layer sizes and randomly determines the network topology 
    (Funnel vs. Flexible).
    """
    # Pool of allowable hidden layer dimensions (Powers of 2)
    # Modify this list to change the search space for layer width.
    pool = [1024, 512, 256, 128, 64, 32]

    # Retrieve the number of layers selected by Ray Tune for this trial
    num_layers = config["num_layers"] 
    
    # Safety Check: Ensure we do not request more layers than available unique sizes in the pool
    # (Prevents errors in random.sample which requires unique elements)
    if num_layers > len(pool):
        num_layers = len(pool)
    # Sample 'num_layers' unique dimensions from the pool
    chosen_dims = random.sample(pool, num_layers) 

    
    # --- Architecture Topology Decision (50/50 Split) ---
    # We introduce randomness to test two different design philosophies:
    if random.choice([True, False]): # 50% di possibilità di essere True
        # OPTION A: FUNNEL (Pyramid) Architecture
        # Sort dimensions in descending order (e.g., 512 -> 128 -> 64).
        # This forces the network to compress information layer by layer (Stability).
        final_dims = sorted(chosen_dims, reverse=True)
    else:
        # OPTION B: FLEXIBLE (Hourglass/Jagged) Architecture
        # Shuffle dimensions randomly (e.g., 128 -> 512 -> 64).
        # This allows the network to expand features in intermediate layers (Exploration).
        random.shuffle(chosen_dims)
        final_dims = chosen_dims
    return final_dims

In [7]:
config = {
    "num_layers": tune.choice([2,3,4,5]),# Randomly select the depth of the Classification Head (2 to 5 layers)
    "hidden_sizes": tune.sample_from(generate_hidden_layer_size),# Generate the specific layer dimensions based on 'num_layers' using the helper function
    "dropout": tune.uniform(0.1, 0.5),# Sample dropout probability uniformly between 10% and 50%
    "lr": tune.loguniform(1e-4, 1e-3),# Learning Rate sampled from a Log-Uniform distribution. We use a conservative range [1e-4, 1e-3] to ensure stable LSTM training.
    "batch_size": tune.choice([10, 20]),# Batch Size: Kept low to strictly prevent OutOfMemory errors on limited hardware
    "num_lstm_layers": tune.choice([1, 2]),# Depth of the Recurrent block (stacked LSTMs)
    "lstm_hidden_size": tune.choice([64, 128]),# Width of the LSTM hidden state (memory capacity)
    "data_ref": data_ref # Pass the lightweight reference to the shared dataset instead of the data itself.
} 

In [8]:
def test_config(config):
    """
    Ray Tune trainable function.
    Executes a 5-Fold Cross-Validation for a specific set of hyperparameters.
    """
    # --- 1. Data Retrieval (Memory Optimized) ---
    # Retrieve the dataset from the Ray Shared Object Store using the reference ID.
    # This avoids serializing and copying the huge dataset to every worker process.
    all_data = ray.get(config["data_ref"])
    
    mcc_scores = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")# Detect hardware availability (GPU/CPU) inside the worker process
    
    # ===========================
    # 5-Fold Cross-Validation Loop
    # ===========================
    for i in range(5):
        # Define split indices: 3 sets for Training, 1 for Validation, 1 for Testing
        # We use a rotating modulo operator to shift sets at every fold.
        training_indices = [(i+1)%5, (i+2)%5, (i+3)%5]
        validation_index = (i+4)%5
        testing_index = i

        # --- 2. Data Preparation ---
        # Extract the X (features) and y (labels) matrices for the training folds
        train_x_list = [all_data[j][0] for j in training_indices]
        train_y_list = [all_data[j][1] for j in training_indices]
        
        # Concatenate the 3 training folds into single contiguous arrays
        x_train_conc = np.concatenate(train_x_list, axis=0)
        y_train_conc = np.concatenate(train_y_list, axis=0)
        
        # Extract Validation and Test sets (no concatenation needed as they are single folds)
        x_val = all_data[validation_index][0]
        y_val = all_data[validation_index][1]
        
        x_test = all_data[testing_index][0]
        y_test = all_data[testing_index][1]

        # --- 3. Dataset & Dataloader ---
        # Initialize the custom Lazy-Loading Datasets
        train_dataset = SignalDataset(x_train_conc, y_train_conc)
        val_dataset = SignalDataset(x_val, y_val)
        test_dataset = SignalDataset(x_test, y_test)

        # Initialize DataLoaders
        # Shuffle is TRUE for training to break data order bias
        train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=config["batch_size"])
        test_loader = DataLoader(test_dataset, batch_size=config["batch_size"])
        
        # --- 4. Model Initialization ---
        model = SP_NN(
            input_size=21, # Fixed at 21 for One-Hot with Explicit Padding
            hidden_sizes=config["hidden_sizes"],
            lstm_hidden_size=config["lstm_hidden_size"],
            num_lstm_layers=config["num_lstm_layers"],
            output_size=1,
            dropout_p=config["dropout"]
        ).to(device) # Move model to GPU/CPU
        
        # Setup Optimizer and Loss Function
        optimizer = optim.Adam(model.parameters(), lr=config["lr"])
        criterion = nn.BCELoss()

        # --- 5. Training Phase ---
        # Run training loop with Early Stopping. 
        best_state = train_val(model, train_loader, val_loader, optimizer, criterion,
                               epochs=100, patience=20)
        
        # Load the weights of the best epoch (not the last one)
        model.load_state_dict(best_state)
        
        # --- 6. Evaluation ---
        # Calculate MCC on the test fold
        mcc = test(model, test_loader)
        mcc_scores.append(mcc)
    # --- 7. Reporting ---
    # Calculate average performance across the 5 folds
    mean_mcc = np.mean(mcc_scores)
    
    # Convert the hidden_sizes list to a string for cleaner logging in the results table
    arch_str = str(config["hidden_sizes"])
    
    # Send metrics back to Ray Tune
    # We report -mean_mcc as 'loss' because Ray minimizes loss by default and if we have negative numbers, 
    #the minimum number will be the higher(in this way we maximize mcc)
    tune.report({"mcc": mean_mcc, "loss": -mean_mcc, "hidden_layers_size": arch_str})

In [9]:
# --- HYPERPARAMETER TUNING EXECUTION ---
result = tune.run(
    test_config,
    config=config,
    num_samples=15, # Total number of different configurations to test

    # Resource Allocation:
    # Assign 1 CPU and 1 GPU to each trial.
    # Since the machine has only 1 GPU, this enforces sequential execution (1 trial at a time),
    # preventing RAM saturation and ensuring full GPU acceleration
    resources_per_trial={"cpu": 1, "gpu": 1}
)


# --- RESULT ANALYSIS ---
# Retrieve the best trial from the experiment history.
# - "mcc": The metric to evaluate.
# - "max": We want to maximize the MCC.
# - "last": We consider the final MCC reported at the end of the cross-validation.
best_trial = result.get_best_trial("mcc", "max", "last")
print("Best trial config:", best_trial.config)
print("Best CV MCC:", best_trial.last_result["mcc"])

2025-11-19 13:37:50,115	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2025-11-19 13:37:50,267	WARNING trial.py:647 -- The path to the trial log directory is too long (max length: 260. Consider using `trial_dirname_creator` to shorten the path. Path: C:\Users\chari\AppData\Local\Temp\ray\session_2025-11-19_13-37-33_443636_16604\artifacts\2025-11-19_13-37-50\test_config_2025-11-19_13-37-50\driver_artifacts\test_config_8f535_00000_0_batch_size=20,dropout=0.2983,lr=0.0010,lstm_hidden_size=128,num_layers=2,num_lstm_layers=1_2025-11-19_13-37-50
2025-11-19 13:37:50,275	WARNING trial.py:647 -- The path to the trial log directory is too long (max length: 260. Consider using `trial_dirname_creator` to shorten the path. Path: C:\Users\chari\AppData\Local\Temp\ray\session_2025-11-19_13-37-33_443636_16604\artifacts\2025-11-19_13-37-50\test_config_2025-11-19_13-37-50\driver_artifacts\test_config_8f535_00000_0_batch_size=20,dropout=0.2983,lr=0.0010,lstm_hidden_size=128,num_layers=2,num_lstm_layers=1_2025-11-19_13-37-50
2025-11-19 13:37:50,287	WARNING trial.py:647 -- Th

Trial name,hidden_layers_size,loss,mcc
test_config_8f535_00000,"[256, 64]",-0.845205,0.845205
test_config_8f535_00001,"[1024, 64]",-0.887224,0.887224
test_config_8f535_00002,"[512, 1024, 32, 128, 256]",-0.849093,0.849093
test_config_8f535_00003,"[512, 64]",-0.86479,0.86479
test_config_8f535_00004,"[128, 1024]",-0.832514,0.832514
test_config_8f535_00005,"[64, 512, 256, 32, 1024]",-0.168544,0.168544
test_config_8f535_00006,"[1024, 256, 128, 64, 32]",-0.904623,0.904623
test_config_8f535_00007,"[512, 64, 256, 32]",-0.844704,0.844704
test_config_8f535_00008,"[1024, 256, 64, 32]",-0.539155,0.539155
test_config_8f535_00009,"[1024, 64, 32]",-0.771861,0.771861


2025-11-19 13:47:55,537	WARNING trial.py:647 -- The path to the trial log directory is too long (max length: 260. Consider using `trial_dirname_creator` to shorten the path. Path: C:\Users\chari\AppData\Local\Temp\ray\session_2025-11-19_13-37-33_443636_16604\artifacts\2025-11-19_13-37-50\test_config_2025-11-19_13-37-50\driver_artifacts\test_config_8f535_00000_0_batch_size=20,dropout=0.2983,lr=0.0010,lstm_hidden_size=128,num_layers=2,num_lstm_layers=1_2025-11-19_13-37-50
2025-11-19 13:48:17,408	WARNING trial.py:647 -- The path to the trial log directory is too long (max length: 260. Consider using `trial_dirname_creator` to shorten the path. Path: C:\Users\chari\AppData\Local\Temp\ray\session_2025-11-19_13-37-33_443636_16604\artifacts\2025-11-19_13-37-50\test_config_2025-11-19_13-37-50\driver_artifacts\test_config_8f535_00001_1_batch_size=20,dropout=0.4441,lr=0.0003,lstm_hidden_size=128,num_layers=2,num_lstm_layers=2_2025-11-19_13-37-50
2025-11-19 13:48:17,411	WARNING trial.py:647 -- Th

Best trial config: {'num_layers': 4, 'hidden_sizes': [256, 128, 64, 1024], 'dropout': 0.4980673167779849, 'lr': 0.00028585527498522286, 'batch_size': 20, 'num_lstm_layers': 2, 'lstm_hidden_size': 128, 'data_ref': ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000001e1f505)}
Best CV MCC: 0.9144333597431926
